<a href="https://colab.research.google.com/github/TrvKhanh/Vietnamese_sentiment/blob/main/Vietnamese_sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install datasets -q
!pip install underthesea -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 106.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.8/657.8 kB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.2 MB/s eta 0:00:00


In [13]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import re
from underthesea import text_normalize, word_tokenize

#**PROCESSING DATA**

In [16]:
ds = load_dataset("sepidmnorozy/Vietnamese_sentiment")
x = ds['train']['text']
def normalize_text(text):
    text = text.lower()
    pattern = r'[^\w\s]'
    new_text = re.sub(pattern, "", text)
    text_clear = text_normalize(new_text)
    return text_clear

from google.colab import drive
drive.mount('/content/drive')

file_path = "/content/drive/MyDrive/vietnamese-stopwords.txt"

def load_stopword(path=file_path):
  try:
    with open(path, 'r', encoding='utf-8') as file:
        stop_words = [line.strip() for line in file]
  except FileNotFoundError:
    print("Error: vietnamese-stopwords.txt not found.")
    stop_words = []
  return stop_words

def remove_stopwords(text, stopwords):
    words = word_tokenize(text)
    filtered_words = [word for word in words if word not in stopwords]
    return " ".join(filtered_words)

def preprocess_text_dataset(dataset):
    dataset = dataset.map(lambda x: {"text": normalize_text(x["text"])})
    dataset = dataset.map(lambda x: {"text": remove_stopwords(x["text"], load_stopword())})
    return dataset

ds = preprocess_text_dataset(ds)

train_dataset = ds['train']
val_dataset = ds['validation']
test_dataset = ds['test']

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Map:   0%|          | 0/2384 [00:00<?, ? examples/s]

Map:   0%|          | 0/331 [00:00<?, ? examples/s]

Map:   0%|          | 0/685 [00:00<?, ? examples/s]

# ***Build Model***

In [26]:
model_base = "vinai/phobert-base"

tokenizer = AutoTokenizer.from_pretrained(model_base)
model = AutoModelForSequenceClassification.from_pretrained(model_base, num_labels=2)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/543M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/543M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=1)
      (position_embeddings): Embedding(258, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         